#### Let's import the nessesary libraries

In [ ]:
import pandas as pd
import math

from multiset import *
from string import punctuation
from datasets import load_dataset_builder
from datasets import load_dataset

from functools import reduce



## The dataset

The following cells will execute the code given in the hugging face tutorial to explore the dataset
and answer the different questions around it

In [ ]:
ds_builder = load_dataset_builder("rotten_tomatoes")

In [ ]:
print(ds_builder.info.description)
ds_builder.info.features

In [ ]:
dataset = load_dataset("rotten_tomatoes")

In [ ]:
pd.Series(dataset["validation"]["label"]).value_counts()

## Naive Bayes classifier 
The next cells will focus on the Naive Bayes classifier.

First we will preprocces the dataset, lowering every character and removing unwanted ponctuation.


In [ ]:
# Let's pre-treat the input by removing the puntuation and lowering it

# We also transfer it into pandas Dataframe format to ease handling 
all_data = {}

def clean_data(text_data):
    text_data = text_data.str.lower()
    text_data = text_data.str.replace("[" + punctuation + "]( |$)", "", regex=True)
    return text_data

for type in dataset.keys():
        
    text_data = pd.Series(dataset[type]["text"])

    text_data = clean_data(text_data=text_data)
    all_data[type] = pd.concat([text_data, pd.Series(dataset[type]["label"])], axis=1)
    all_data[type].columns=["text", "label"]
    
    
data = all_data["train"]
data

We tokenize words with a multiset data structure (bag of word method)

In [ ]:
def find_vocabulary(text_data: pd.DataFrame):
    td = text_data.copy()
    td["text"] = td["text"].apply(lambda t: Multiset(t.split(" ")))
    
    return td.groupby(by="label")["text"].sum()

V_c = find_vocabulary(data)
V = V_c.sum()

C = [0, 1]

We implement the Train Naive Bayes function dicribed in the lecture

In [ ]:
def train(data: pd.DataFrame, classes: list):
    V_c = find_vocabulary(data) # called bigdoc in pseudo code
    V = V_c.sum()
    
    # creates empty df where we will store our word occurence conditional probabilities 
    logLikelihood = pd.DataFrame(index=pd.Index(set(V)), columns=classes) 

    logprior = {}
    n_data = len(data)
    for c in classes:
        n_c = data["label"].value_counts()[c]
        logprior[c] = math.log(n_c / n_data)
        
        cardinal_V = (len(V) + len(V.items()))
        
        for w in V:
            count = V_c[c][w]
            logLikelihood.loc[w, c] = math.log((count + 1) / cardinal_V)
            
    return logprior, logLikelihood
            


logprior, logLikelihood = train(data, C)

In [ ]:
predict_args = (logprior, logLikelihood, C, V)

#Implements Test Naive Bayes function
def predict(doc, logprior, logLikelihood, C, V):
    sumed = {}
    for c in C:
        sumed[c] = logprior[c]
        for w in doc.split(" "):
            if w in V:
                sumed[c] += logLikelihood.loc[w, c]

    return max(sumed, key=sumed.get)

# Applies prediction to a whole df
def predict_dataset(data, logprior, logLikelihood, C, V):
    data["results"] = (
        data["text"].apply(lambda t: predict(t, logprior, logLikelihood, C, V))
        == data["label"]
    )

# Print the percent of right answer
def evaluate(data, name, logprior, logLikelihood, C, V):
    print(f"results on {name} dataset : ", data["results"].value_counts()[True] / len(data))

# Predicts and evaluate on all our dataset
for type in dataset.keys():
    predict_dataset(all_data[type], *predict_args)   
    evaluate(all_data[type], type, *predict_args)

Next cells build the Naive Bayes Classifier using sklearn

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

estimator = [('Cv', CountVectorizer()), ('Bayes', MultinomialNB())]
pipe = Pipeline(estimator)
pipe.fit(data.text, data.label)

In [ ]:
for type in dataset.keys():  
    print(f"{type} : {pipe.score(all_data[type]['text'],all_data[type]['label'])}")


**Most likely, the scikit-learn implementation will give better results. Looking at the documentation, explain why it could be the case.**

Sklearn seems better because it uses a Laplace smoothing parameter.

To test this hypotheis I run the same model but with no smoothing parameter

In [ ]:
estimator = [('Cv', CountVectorizer()), ('Bayes', MultinomialNB(alpha=0, force_alpha=True))]
pipeNoalpha = Pipeline(estimator)
pipeNoalpha.fit(data.text, data.label)

for type in dataset.keys():  
    print(f"{type} : {pipe.score(all_data[type]['text'],all_data[type]['label'])}")


The accuracy droped on test and was higher on train, the model seems to overfit.

The smoothing parameter is definitely important to get a better accuracy

**Why is accuracy a sufficient measure of evaluation here?**

It is a sufficient measure because the classes or equally split.

In [ ]:
prediction = pipe.predict(all_data["train"]['text'])

mask = prediction != all_data['train']['label']
# convert the mask to a pandas series
series = pd.Series(mask)

series.value_counts()

In [ ]:
wrong = all_data['train'].loc[series]

def printWrong(row : pd.Series):
    print(f"We were wrong on  : {row['text']}\nReal label : {'positive' if row['label'] else 'negative'}")

wrong.head().apply(printWrong, axis=1)

None


In [ ]:
wrong

### 8. [Bonus] What are the top 10 most important words (features) for each class?

In [ ]:
for c in [0, 1]:
    print(f"Most impactfull words in class {c} :")
    display(logLikelihood.sort_values(by=c, ascending=False)[c].head(10))

In [ ]:
# Imports the stopword list
import ssl
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))
stops.add('')
print(stops)


In [ ]:

for c in [0, 1]:
    print(f"Most impactfull words in class {c} (exepting stopwords):")
    most_impact = logLikelihood.sort_values(by=c, ascending=False)[c]
    display(most_impact.loc[~most_impact.index.isin(stops)].head(10))


Now let's look at scikit's log_probs

In [ ]:

names = pd.Series(pipe["Cv"].get_feature_names_out(), name="name")
scikit_log_prob = pd.DataFrame(pipe["Bayes"].feature_log_prob_).transpose()
scikit_log_prob.columns = pipe["Bayes"].classes_
scikit_log_prob.index = names

for c in pipe["Bayes"].classes_:
    print(f"Most impactfull words in class {c} in scikit learn :")
    most_impact = scikit_log_prob.sort_values(by=c, ascending=False)[c]
    display(most_impact.head(10))

    print("And without stop words :")

    display(most_impact.loc[~most_impact.index.isin(stops)].head(10))

### 9. [Bonus] Play with scikit-learn's version parameters. For example, see if you can consider unigram and bigram instead of only unigrams.w